## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [ ]:
import pandas as pd
from pymystem3 import Mystem

In [25]:
data = pd.read_csv('/Users/alexey_zalesov/Desktop/ya_prakrikum/ds/datasets/data.csv')
data.info()
print()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   

### Вывод
Одинаковые по смыслу значения в столбце 'purpose' различаются по написанию ("сыграть свадьбу" и "на проведение свадьбы"); есть одинаковые значения, различающиеся по регистру. Всего есть данные о 21525 клиентах, а ненулевых строчек о занятости и общем доходе 19351 - соответственно есть пропуски. Есть отрицательные значения в столбце "days_employed". 

## Предобработка данных

### Обработка пропусков

In [26]:
print(data.isnull().sum())
#вместо среднего арифметического меняем на медиану 
mean_days_empoyed = data['days_employed'].median()
mean_total_income = data['total_income'].median()
data['days_employed'] = data['days_employed'].fillna(mean_days_empoyed)
data['total_income'] = data['total_income'].fillna(mean_total_income)
print()
data.info() #проверка, что пропусков больше нет

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_in

### Вывод
В столбцах 'children', 'dob_years', 'education_id', 'income_type', 'debt', пропусков нет. 
В столбце 'gender' есть странное значение 'XNA' - возможно, ошибка выгрузки. 
Пропуски имеются в 'days_employed' и 'total_income'. Их 2174 - слишко много, чтобы просто удалить. Так как количество пропусков в обоих столбцах совпадает, скорее всего, пропуски связаны между собой: нет информации о стаже - нет дохода. Пропуски были заменены на средние арифметические соответствующих столбцов.

### Замена типа данных

In [27]:
data['total_income'].astype('int')
data['days_employed'].astype('int')

0         -8437
1         -4024
2         -5623
3         -4124
4        340266
          ...  
21520     -4529
21521    343937
21522     -2113
21523     -3112
21524     -1984
Name: days_employed, Length: 21525, dtype: int64

### Вывод
Значения обоих столбцов 'total_income' и 'days_employed' заменены на целочисленные. В столбце 'days_employed' есть отрицательные значения, но так как данный столбец не нужен нам для ответа на поставленный вопрос - он был оставлен без изменений. Так как необходимо заменить вещественный тип данных на целочисленный, и погрешность, получаемая при изменении типа мала, импользуется метод astype().

### Обработка дубликатов

In [28]:
data['education'].str.lower()
data['gender'].str.lower()
data['income_type'].str.lower()
data['purpose'].str.lower()
data.duplicated().sum() #проверяем количество дупликатов
data.drop_duplicates().reset_index(drop=True) #удаляем дупликаты 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21468,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21469,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод
Так как всего 54 дубликата, а это очень малая доля всех данных - можно просто удалить методом drop_duplicates(). Так как количество дубликатов очень мало, скорее всего, дубликаты появились в процессе выгрузки или внесения информации. 

### Лемматизация

In [29]:
from pymystem3 import Mystem
m = Mystem()

lemmas_list = []
def lemmatize(data):
    lemmas = ''.join(m.lemmatize(data)).strip("\n")
    lemmas_list.append(lemmas)
    return lemmas
    
data['lemmatized_purpose'] =  data['purpose'].apply(lemmatize)
#lemmas_dict=set(lemmas_list)
#print(lemmas_dict) #смотрим леммы

def purpose_category(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'строительство' in row:
        return 'строительство'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'ремонт' in row:
        return 'ремонт'
    if 'образование' in row:
        return 'образование'
    if 'жилье' in row:
        return 'недвижимость'
    pass


#создаем новый столбец с категориями целей получения кредита
data['categorized_purpose'] =  data['lemmatized_purpose'].apply(purpose_category) 
print(data.head())

print(data['categorized_purpose'].unique())





   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose          lemmatized_purpose categorized_purpose  
0               покупка жилья             

### Вывод
Созданы два новых столбца, проведена лемматизация, а после - категоризация по целям (строительство недвижимости, получение образование, ремонт (автомобиля или жилья), проведение свадьбы, покупка автомобиля). 

### Категоризация данных

In [30]:
#создадим столбец kids_status - значение 1, если дети есть и 0, если их нет
def kids_status(kids):
    a=1
    b=0
    if kids >= 1:
        return a
    else:
        return b
data['kids_status'] = data['children'].apply(kids_status)
#print(data.head())
#создадим столбец marriage_status, 0 - не женат или не замужем, 1 - гражданский брак, 2 - женат или замежум, 3 - в разводе, 4 - вдовец или вдова. 
data['family_status'].value_counts()
def marriage_status(status):
    a = 0
    b = 1
    c = 2
    d = 3
    e = 4
    if status == 'женат / замужем':
        return 2
    if status == 'гражданский брак':
        return 1
    if status == 'Не женат / не замужем':
        return 0
    if status == 'в разводе':
        return 3
    else:
        return 4
data['marriage_status'] = data['family_status'].apply(marriage_status)
#создадим столбец income_status, категоризирующий данные по уровню дохода по 
data['total_income'].quantile([.25, .5, .75]) #находим границы квантилей
#0.25.   107798.172619

#0.50    156400.319836
#0.75    195543.620942

def income_type(income):
    if income<=107798.172619:
        return 'small'
    if 107798.172619<income<=156400.319836:
        return 'average'
    if 156400.319836<income<=195543.620942:
        return 'high'
    else:
        return 'top'

data['income_status'] = data['total_income'].apply(income_type)
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose          lemmatized_purpose categorized_purpose  \
0               покупка жилья            

### Вывод
Категоризация проведена для ответа на поставленные в дальнейшем вопросы. Выделены "словари" данных заемщиков с детьми и без, по типу семейного положения, разными уровнями дохода и целями получения кредита. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
#для ответа на этот вопрос сравним соотношение должников с детьми и без детей к общему количеству должников
#общее количество должников
data[data['debt']==1].count() 
total_debtors = 1741
#общее количество должников, у которых есть дети
data[(data['debt']==1) & (data['kids_status']==1)].count()
debtors_with_kids = 677
#получается, должников без детей: 1741-677=1064
debtors_without_kids = 1064

debtors_part_with_kids = debtors_with_kids / total_debtors
debtors_part_without_kids = debtors_without_kids / total_debtors
#print(debtors_part_with_kids, 'Доля должников с детьми')
#print(debtors_part_without_kids, 'Доля должников без детей')


df_kids = data.pivot_table(index = 'debt', columns = 'kids_status', values = 'education', aggfunc = 'count'  )
df_kids = df_kids.transpose()
df_kids['total_clients'] = df_kids[1] + df_kids[0]
df_kids['debt_of_all_clients'] = df_kids[1] / df_kids['total_clients']
df_kids = df_kids.transpose()
print(df_kids)


kids_status                     0            1
debt                                          
0                    13132.000000  6652.000000
1                     1064.000000   677.000000
total_clients        14196.000000  7329.000000
debt_of_all_clients      0.074951     0.092373


### Вывод
Так как доля должников с детьми значительно меньше доли должников без детей, можно сказать, что заемщик, имеющий детей, скорее вернет кредит, чем заемщик, не имеющий детей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
#категории: 0 - не женат или не замужем, 
#1 - гражданский брак, 2 - женат или замужем, 3 - в разводе, 4 - вдовец или вдова. 
data[(data['debt']==1) & (data['marriage_status']==0)].count()
bachelors_debtors = 274
data[(data['debt']==1) & (data['marriage_status']==1)].count()
civil_marriage_debtors = 388
data[(data['debt']==1) & (data['marriage_status']==2)].count()
married_debtors = 931
data[(data['debt']==1) & (data['marriage_status']==3)].count()
divorced_debtors = 85
data[(data['debt']==1) & (data['marriage_status']==4)].count()
widow_debtors = 63

part_b_d = bachelors_debtors / total_debtors
part_c_d = civil_marriage_debtors / total_debtors
part_m_d = married_debtors / total_debtors
part_d_d = divorced_debtors / total_debtors
part_w_d = widow_debtors / total_debtors

print(part_b_d, 'доля должников, которые не женаты/не замужем')
print(part_c_d, 'доля должников, которые в гражданском браке')
print(part_m_d, 'доля должников, которые женаты')
print(part_d_d, 'доля должников в разводе')
print(part_w_d, 'доля должников, которые овдовели')

data_marriage_debt = data.pivot_table(index = 'debt', columns = 'marriage_status', values = 'education', aggfunc = 'count'  )
data_marriage_debt = data_marriage_debt.transpose()
data_marriage_debt['total_clients'] = data_marriage_debt[1] + data_marriage_debt[0]
data_marriage_debt['debt_of_all_clients'] = data_marriage_debt[1] / data_marriage_debt['total_clients']
data_marriage_debt = data_marriage_debt.transpose()
print(data_marriage_debt)


0.15738081562320505 доля должников, которые не женаты/не замужем
0.22286042504307868 доля должников, которые в гражданском браке
0.5347501435956347 доля должников, которые женаты
0.048822515795519814 доля должников в разводе
0.03618609994256174 доля должников, которые овдовели
marriage_status                0           1             2           3  \
debt                                                                     
0                    2539.000000  3789.00000  11449.000000  1110.00000   
1                     274.000000   388.00000    931.000000    85.00000   
total_clients        2813.000000  4177.00000  12380.000000  1195.00000   
debt_of_all_clients     0.097405     0.09289      0.075202     0.07113   

marriage_status               4  
debt                             
0                    897.000000  
1                     63.000000  
total_clients        960.000000  
debt_of_all_clients    0.065625  


### Вывод
Зависимость между семейным положением и возвратом кредита в срок есть - доли должников с различным семейным статусом значимо различаются. Скорее всего, заемщики без партнера несут меньше рисков и имеют меньше обязанностей, чем заемщики с партнером, и тем более заемщики, состоящие в браке. Разведенные люди, скорее всего, старше первых трех групп, соответственно скорее имеют более качественное образование и несут меньше рисков - у них больше определенности. Овдовевшие люди, скорее всего, еще старше и несут еще меньше рисков. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:

data[(data['debt']==1) & (data['income_status']=='top')].count()
debtors_top_income = 386
data[(data['debt']==1) & (data['income_status']=='high')].count()
debtors_high_income = 456
data[(data['debt']==1) & (data['income_status']=='average')].count()
debtors_average_income = 472
data[(data['debt']==1) & (data['income_status']=='small')].count()
debtors_small_income = 427

print(debtors_top_income / total_debtors, 'доля должников с самым высоким уровнем дохода')
print(debtors_high_income / total_debtors, 'доля должников с  высоким уровнем дохода')
print(debtors_average_income / total_debtors, 'доля должников со средним уровнем дохода')
print(debtors_small_income / total_debtors, 'доля должников с маленьким уровнем дохода')

df_income_level = data.pivot_table(index = 'debt', columns = 'categorized_purpose', values = 'education', aggfunc = 'count')
df_income_level = df_income_level.transpose()
df_income_level['total_clients'] = df_income_level[1] + df_income_level[0]
df_income_level['debt_of_all_clients'] = df_income_level[1] / df_income_level['total_clients']
df_income_level = df_income_level.transpose()
print(df_income_level)

0.22171165996553704 доля должников с самым высоким уровнем дохода
0.2619184376794945 доля должников с  высоким уровнем дохода
0.27110855829982766 доля должников со средним уровнем дохода
0.24526134405514072 доля должников с маленьким уровнем дохода
categorized_purpose   автомобиль  недвижимость  образование     ремонт  \
debt                                                                     
0                    3912.000000   7744.000000  3652.000000  577.00000   
1                     403.000000    603.000000   370.000000   35.00000   
total_clients        4315.000000   8347.000000  4022.000000  612.00000   
debt_of_all_clients     0.093395      0.072242     0.091994    0.05719   

categorized_purpose      свадьба  строительство  
debt                                             
0                    2162.000000    1737.000000  
1                     186.000000     144.000000  
total_clients        2348.000000    1881.000000  
debt_of_all_clients     0.079216       0.076555  


### Вывод
Долг скорее вернет человек из первого или последнего квантиля по доходам. Возможно, это связано с тем, что им проще всего сохранить свой уровень дохода, в то время как "середине" сложнее, и она вероятней "потеряет" текущий уровень дохода и не сможет выполнить свое обязателство по уровню дохода. 

- Как разные цели кредита влияют на его возврат в срок?

In [34]:
#сравним доли должников с разными целями
data[(data['debt']==1) & (data['categorized_purpose']=='свадьба')].count()
marriage_payment = 186
data[(data['debt']==1) & (data['categorized_purpose']=='автомобиль')].count()
buy_car = 403
data[(data['debt']==1) & (data['categorized_purpose']=='ремонт')].count()
repair_payment = 35
data[(data['debt']==1) & (data['categorized_purpose']=='образование')].count()
education_payment = 370
data[(data['debt']==1) & (data['categorized_purpose']=='строительство')].count()
building_estate = 144
data[(data['debt']==1) & (data['categorized_purpose']=='недвижимость')].count()  
estate_purchase = 603

print(estate_purchase/total_debtors, 'доля должников с целью покупки недвижимости')
print(buy_car/total_debtors, 'доля должников с целью покупки личного автомобиля')
print(building_estate/total_debtors, 'доля должников с целью строительства недвижимости')
print(repair_payment/total_debtors, 'доля должников с целью ремонта')
print(marriage_payment/total_debtors, 'доля должников с целью проведения свадьбы')
print(education_payment/total_debtors, 'доля должников с целью улучшения образования')

df_goals = data.pivot_table(index = 'debt', columns = 'categorized_purpose', values = 'education', aggfunc = 'count')
df_goals = df_goals.transpose()
df_goals['total_clients'] = df_goals[1] + df_goals[0]
df_goals['debt_of_all_clients'] = df_goals[1] / df_goals['total_clients']
df_goals = df_goals.transpose()
print(df_goals)

0.3463526708788053 доля должников с целью покупки недвижимости
0.231476163124641 доля должников с целью покупки личного автомобиля
0.08271108558299828 доля должников с целью строительства недвижимости
0.020103388856978748 доля должников с целью ремонта
0.10683515221137277 доля должников с целью проведения свадьбы
0.2125215393452039 доля должников с целью улучшения образования
categorized_purpose   автомобиль  недвижимость  образование     ремонт  \
debt                                                                     
0                    3912.000000   7744.000000  3652.000000  577.00000   
1                     403.000000    603.000000   370.000000   35.00000   
total_clients        4315.000000   8347.000000  4022.000000  612.00000   
debt_of_all_clients     0.093395      0.072242     0.091994    0.05719   

categorized_purpose      свадьба  строительство  
debt                                             
0                    2162.000000    1737.000000  
1                     186.

### Вывод
Самые невозвращаемые цели получения кредита: покупка недвижимости, покупка автомобиля, улучшение образования. Возможно, это связанно с тем, что это достаточно крупные кредиты, которые клиентам тяжелее вернуть. Ремонт, строительство и проведение свадьбы не возвращаются гораздо реже, скорее всего, это меньшие траты и заемщикам проще их вернуть. 

### Шаг 4. Общий вывод

Выделены наиболее опасные признаки, которые указывают на высокую вероятность невозврата кредита заемщиком. Благодаря анализу предоставленных данных, можно внести изменения в условия предоставления займа и снизить количество невозвращаемых кредитов.  

### Чек-лист готовности проекта


- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [X]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.